In [1]:
import torch
from time import time
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from zipfile import ZipFile
import numpy as np
import pandas as pd
from time import time
import zipfile
import re
import os
import torchvision
from torchvision import datasets
from torchvision import transforms
import matplotlib.pyplot as plt
from PIL import Image
from torchvision.io import read_image
from matplotlib import image

In [2]:
from vgg_pretrained import vgg_all_freezed,total_trainable_parameters,full_vgg,vgg_layer_freeze
from prepare_dataset import MyDataset,subset_generator
from vgg_pretrained import vgg_all_freezed,total_trainable_parameters,full_vgg,vgg_layer_freeze
from train_fn import train,validation,image_validation,validation_classes,train_batch
from Model import Network,new_model,new_Regressor

In [3]:
fullvgg=full_vgg()

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
transform = transforms.Compose([transforms.ToTensor(),normalize])
target_transform=transforms.Compose([transforms.ToTensor()])


In [4]:
#will take lot of time due to opening each file and creating the dataframe
anno_dir=r"C:\Users\Engineer Protobetz\Documents\Programs Directory\data\AffectNet\train_set\annotations"
image_dir=r"C:\Users\Engineer Protobetz\Documents\Programs Directory\data\AffectNet\train_set\images"
train_data=MyDataset(anno_dir,image_dir,transform=transform, target_transform=None)


This dataset is done



In [5]:

val_anno_dir=r"C:\Users\Engineer Protobetz\Documents\Programs Directory\data\AffectNet\val_set\annotations"
val_image_dir=r"C:\Users\Engineer Protobetz\Documents\Programs Directory\data\AffectNet\val_set\images"
val_data=MyDataset(val_anno_dir,val_image_dir,transform=transform, target_transform=None)

This dataset is done



In [6]:
batch = 64
train_loader=torch.utils.data.DataLoader(train_data,
                                         batch_size=batch,
                                         shuffle=True,
                                         num_workers=4,
                                         pin_memory=True,
                                         sampler=None)
val_loader=torch.utils.data.DataLoader(val_data,
                                       batch_size=batch,
                                       shuffle=False,
                                       num_workers=4,
                                       pin_memory=True,
                                      sampler=None)
val_class_loader=torch.utils.data.DataLoader(val_data,
                                       batch_size=1,
                                       shuffle=False,
                                       num_workers=4,
                                       pin_memory=True,
                                      sampler=None)

In [ ]:
df = train_data.basis_to_exp()
X = np.array(df.loc[:, ~df.columns.isin(['class'])])
y = np.array(df['class'])
print(X)
print(y)

In [7]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


In [8]:
feature_extractor=vgg_layer_freeze(17)
total_trainable_parameters(feature_extractor)


Total trainable parameters of Sequential is 	 12979200


In [9]:

#input_features_for_denselayer=get_output_features_of_model(model=feature_extractor,batch_size=32,dataloader=train_loader)
input_features_for_denselayer=25088


In [10]:
modelReg=new_Regressor(feature_extractor,input_features_for_denselayer).to(device)
total_trainable_parameters(modelReg)


Total trainable parameters of new_Regressor is 	 222704642


In [11]:
loss_plain=nn.L1Loss()
optimizer=optim.NAdam(modelReg.parameters(),lr=0.0001)
classes = ['Valance', 'Arousal']
classes_exp=['Neutral','Happy','Sad','Surprise','Fear','Disgust','Anger','Contempt']

In [12]:
epochs = 5
for epoch in range(epochs):
    print(f'Epoch-{epoch}')
    train_batch(train_loader,modelReg,loss_plain,optimizer)
    validation(val_loader,modelReg,loss_plain)

Epoch-0


  0%|                                                                          | 1/287651 [00:14<1176:20:19, 14.72s/it]

loss:0.055920 batch:0/287651
loss:0.039442 batch:0/287651


  0%|                                                                           | 3/287651 [00:18<385:22:55,  4.82s/it]

loss:0.152717 batch:2/287651
loss:0.030995 batch:2/287651


...

loss:0.007087 batch:32/287651
loss:0.043083 batch:32/287651


  0%|                                                                          | 35/287651 [01:22<172:35:48,  2.16s/it]

loss:0.012443 batch:34/287651
loss:0.013449 batch:34/287651


  0%|                                                                          | 36/287651 [01:26<191:54:57,  2.40s/it]


KeyboardInterrupt: 

In [ ]:
from tqdm import tqdm
def train12(dataloader,model,loss_fn,optimizer):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model.train()
    for batch,(X,y_exp,y_val,y_aro) in enumerate(tqdm(dataloader)):
        y_val = y_val.type(torch.LongTensor).view(-1,1)
        y_aro = y_aro.type(torch.LongTensor).view(-1,1)
        X,y_val,y_aro=X.to(device),y_val.to(device),y_aro.to(device)

        predVal, predAro = model(X)
        print(predVal.shape)
        print(predAro.shape)
        print(y_val.shape)
        print(y_aro.shape)
        break
        """loss1=loss_fn(pred,predVal)
        loss2=loss_fn(pred,predAro)

        optimizer.zero_grad()
        loss1.backward()
        loss2.backward()
        optimizer.step()
        """
train12(train_loader,modelReg,loss_plain,optimizer)

In [ ]:
"""
Trains the val and arousal to give the emotion class
The depth is limited to 6 to prevent overfitting and on recognizing complexity from human perspective
The training is independant of the NN but will take the output of the NN to get the emotion class
The dataset X, y will take a lot of time due to the 
"""

from xgboost import XGBRFClassifier

# using random forest due to visible relation
clf = XGBRFClassifier(objective='multi:softmax')

n_estimators = [10,100, 200]
learning_rate = [0.1, 1]
booster = ['dart']
tree_method = ['hist']
eval_metric = ['auc']

gamma = [0.1, 1, 5]
#l1
reg_alpha = [0.1,1,10]
#l2 lambda required
reg_lambda = [0.1,1,10]
n_jobs = [-1]
subsample = [0.33]
max_depth = [6]


grid_state = {'n_estimators':n_estimators,
              'learning_rate':learning_rate,
              'booster':booster , 
              'tree_method':tree_method,
              'n_jobs':n_jobs,
              'eval_metric':eval_metric,
              'reg_alpha':reg_alpha,
              'reg_lambda':reg_lambda,
              'gamma': gamma,
              'subsample':subsample,
              'max_depth':max_depth
             }

xgb_grid = GridSearchCV(estimator = clf,
                        param_grid = grid_state,
                        scoring = "roc_auc",
                        cv=5,
                        verbose=0,
                    )
xgb_grid.fit(X,y)

In [ ]:
def predict(clf, reg):
    # to write once someone has the NN model
    